# Laravel

In [ ]:
$ composer create laravel/laravel <projectname>

In [ ]:
php artisan serve

### Connecting to react app
After initialising your laravel project:

In [ ]:
$ npm install
$ npm run dev ## Laravel mix compiles resources/js/app

$ npm install --save-dev react react-dom

Update the Laravel Mix config to compile react:

In [ ]:
## webpack.mix.js

const mix = require('laravel-mix');

mix.js('resources/js/app.js', 'public/js')
    .react()
    .postCss('resources/css/app.css', 'public/css', [
        //
    ]);

Can run npm run watch so that it automatically compiles files upon change

In [ ]:
$ npm run watch

## Basics

### Routing
Php route files load view files.

In [ ]:
## Routes/web.php

# Calls resources/views/posts.blade.php
Route::get('/', function ()) {
    return view('posts')
}
    


### Javascript files
Below view calls our app.js javascript file, which loads the relevant components. We can then call javascript functions via the id of a given div.

In [ ]:
## Resources/views/posts.blade.php
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Laravel React</title>
    <link rel="stylesheet" href="/app.css">

    <script src="{{ asset('js/app.js') }}" defer></script>
</head>
<body>
    <div id="allBlogs"></div>
</body>
</html>

In [ ]:
## Resources/js/app.js
require('./bootstrap');

require('./components/AllBlogs');
require('./components/Post')

In [ ]:
## Resources/js/components/AllBlogs
import React from 'react'
import ReactDOM from 'react-dom'
import BlogItem from './Blog_list_item'

export default function AllBlogs(){

    const blog = {
        title: "Kooks is very beautiful",
        date: new Date()
    }
    return (
        <div>
            <h1>Blogs</h1>
            <ul>
                <BlogItem blog={blog}/>
            </ul>
        </div>
    )
}

if (document.getElementById('allBlogs')){
    ReactDOM.render(<AllBlogs/>, document.getElementById('allBlogs'))
}

### React communicating with server

In [ ]:
// Inside react component
import axios from "axios"

const notifyServer = () => {
    axios.post('/route', {
        message: 'Info sending'
    })
}

### Passing arguments to react components
Can send variables in associative arrays.

In [ ]:
## routes/web.php

Route::get('post', function () {
    return view('post', [
        'post' => "Hello world!"
    ]);
});

Then pass the variable to the php view

In [ ]:
## resources/views/post.blade.php
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Laravel React</title>

    <script src="{{ asset('js/app.js') }}" defer></script>
</head>
<body>
    <div id="post" post="{{ $post }}"></div>
</body>
</html>

Define variable in react component

In [ ]:
import React from 'react'
import ReactDOM from 'react-dom'

export default function Post({post}) {
    console.log(post)
    return (
        <div>
            <h1>{post}</h1>
            <b/>
            <a href = "/">back</a>
        </div>
    )
}

if (document.getElementById('post')){
    const value = document.getElementById('post').getAttribute("post")
    ReactDOM.render(<Post post={value}/>, document.getElementById('post'))
}

### Taking arguments from route uri

In [ ]:
## routes/web.php
Route::get('posts/{post}', function ($slug){
    
    if (!file_exists($path = __DIR__ . "resources/{$post}.html")) {
        abort(404)
        ## Can also redirect
        return redirect('/')
    }
    $post = file_get_contents($path);
    return view('post', [
        'post' => $post
    ]);
})->where('post', '[a-z]+'); # Where specifies route only works for certain regex-validated variables

### Passing JSONs

In [ ]:
Route::get('/post/{id}', function ($id) {
    $blogs = json_decode(file_get_contents(__DIR__ . "/../resources/data/posts.json"));
    $blog = Arr::where($blogs->blogs, function ($value, $key) use ($id) {
        return $key == $id;
    });
    die(var_dump($blog));
});

### Caching
Laravel provides easy helper functions for storing cache. Below route saves cache for one minute. now()->addXXX works for many different periods of time. Forgets cache after 1 minute. 

In [ ]:
Route::get('/post/{id}', function ($id) {
    ///...
    
    $blogs = cache()->remember("post.{$id}", now()->addMinutes(1), function () use ($path, $id){
        $blogs = json_decode(file_get_contents($path));
        return Arr::where($blogs->blogs, function ($value, $key) use ($id) {
            return $key == $id;
        });
    });
})

### Refactoring 
Suggested refactoring to make code nicer. Write down exactly what you want the function to do. Then rewrite the code so it reads just as that.

Make a new class in *app/models**

In [ ]:
use App\Models\Blog;

Route::get('/post/{id}', function ($id) {
    $blog = Blog::find($slug);
    
    return view('post', [
        'blog' => json_encode($blog[$id])
    ]);
})

Name space maps to directory names and it used for accessing different files and objects. 

In [ ]:
<?php 

namespace App\Models;
use Illuminate\Database\Eloquent\ModelNotFoundException;
use Illuminate\support\Arr;

class Blog 
{
    public static function find($id)
    {
        if (!file_exists($path = resource_path("data/posts.json"))){
            return ModelNotFoundException(); # Laravel helper throws error if not found
        }
    
        return cache()->remember("post.{$id}", now()->addMinutes(1), function () use ($path, $id){
            $blogs = json_decode(file_get_contents($path));
            return Arr::where($blogs->blogs, function ($value, $key) use ($id) {
                return $key == $id;
            });
        })[$id];
    }
}

## Connecting to a DB
In *config/database.php* find a variable 'default' which defines the default DB connection. Populates all .env variables.

In [ ]:
'default' => env('DB_CONNECTION', 'mysql')  ## Default to mysql if none

Create DB in mysql and run php artisan migrate in shell.

In [ ]:
$ php artisan migrate

*database/migrations* contains a load of schemas for columns in the DB tables. Need to migrate each time you make a change to the schema.

In [ ]:
$ php artisan migrate::rollback ## rollbacks last migration
$ php artisan migrate  # migrates

In [ ]:
class CreateUsersTable extends Migration
{
    /**
     * Run the migrations.
     *
     * @return void
     */
    public function up()
    {
        Schema::create('users', function (Blueprint $table) {
            $table->id();
            $table->string('name');
            $table->string('email')->unique();
            $table->timestamp('email_verified_at')->nullable();
            $table->string('password');
            $table->boolean('is_admin')->default(false); ## Set defaults
            $table->rememberToken();
            $table->timestamps();
        });
    }

    /**
     * Reverse the migrations.
     *
     * @return void
     */
    public function down()
    {
        Schema::dropIfExists('users');
    }
}

In [ ]:
$ php artisan migrate:rollback ## rollbacks last migration
$ php artisan migrate:fresh  ## rollsback EVERY migration

In [ ]:
$ php artisan ## shows all command options

### Eloquent models
Eloquent is laravel's object relational mapper (way of interacting with DB tables). Each table has a corresponding eloquent model (Users table -> User model), and each instance of a model corresponds to a single row in that table. This uses the **Active Record Pattern**: An approach to accessing data in DB where each object instance of a model is tied to a single row in the table. 

Can use php artisan tinker to add rows to table from the command line.

In [ ]:
$ php artisan tinker 

> $user = new App\Models\User; # use name space of model
> $user->name = "Kooks"
> $user->email = "kooks@kooks.com"
> $user->password = bcrypt('password')  ## use bcrypt to encrypt password

> $user->save();

Can always edit user info by re-defining any of these and then re-saving. 

In [ ]:
> User::find(1);  # find user with id of 1, returns null if none
> User::findOrFail(1); # returns modelnotfound exception if none
        
> User:all(); # get colleciton of all users
> User::count()
## Save collection to variable
> $users = User:all();
    
> $users->pluck('name'); # get new collection of names
> $users->first(); # get first item from collection

Can also use collections as an array!

### Making eloquent models

Quickly, at any point can use php artisan help <command> to see which arguments it takes. 

To make a new table, want to start by defining its migration:

In [ ]:
$ php artisan make:migration create_posts_table  ## creates migration file in database/migrations

Then you can edit the schema accordingly. 

Then make a model using the singular version of the table name.

In [ ]:
$ php artisan make:model Post

Alternatively you can run this to execute both at once: 

In [ ]:
$ php artisan make:model Post -m ## -m includes migration

Make a new post

In [ ]:
$ use App\Models\Post;
Post::all();

### Mass Assignments
Can make an object instance all at once by defining $fillable properties in your model. In general only want to allow mass assignment (ie set to fillable) of variables you would like your users to be able to edit. Ie. not things like ID. These are called Mass Assignment Vulnerabilities. 

In [ ]:
## app\Models\Post
<?php
namespace App\Models;

class Post extends Model
{
    protected $fillable = ["title", 'excerpt']
}

In [ ]:
> Post::create(['title'=> 'blog1', 'excerpt'=> 'a compelling new blog post']);

### DB seeding
Can seed the database to automatically fill with data after each migration. 

In [ ]:
# database/seeds/DatabaseSeeder.php
use App\Models\Category;
use App\Models\Blog;
use Illuminate\Database\Seeder;

class DatabaseSeeder extends Seeder
{
    /**
     * Seed the application's database.
     *
     * @return void
     */
    public function run()
    {
        Category::truncate();
        Blog::truncate();

        $fun = Category::create([
            'name'=> 'Fun',
            'slug' => 'fun'
        ]);

        Blog::create([
            'category_id'=> $fun->id,
            'title'=>"blog post 1",
            'author' => 'kooks',
            'excerpt' => 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.',
            'content'=> 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.'
        ]);

    }
}

In [ ]:
$ php artisan db:seed

### Factories
Can use factories to seed database

In [ ]:
? php artisan make:factory PostFactory
    
### OR when making a new model:
?php make:model -a # makes migration, seeder, factory and resource controller!

In [ ]:
## database/factories/PostFactory
<?php

namespace Database\Factories;

use Illuminate\Database\Eloquent\Factories\Factory;

class BlogFactory extends Factory
{
    /**
     * Define the model's default state.
     *
     * @return array
     */
    
    protected $model = Blog::class; 

    public function definition()
    {
        return [
            'user_id'=> User::factory(), # automatically finds user ID from factory
            'category_id'=> Category::factory(),
            'title'=> $this->faker->sentence,
            'excerpt' => $this->faker->sentence,
            'content'=>$this->faker->paragraph
        ];
    }
}

And in the model, need to make sure to mark it as "has factory":

In [ ]:
# App\Models\Post

use Illuminate\Database\Eloquent\Factories\HasFactory;

class Post extends Model {
    use HasFactory;
    //...
}

In [ ]:
> $blog = Blog::factory()->create();

## Specify category ID
> $blog = Blog::factory()->create('category_id' => 1)

Automatically seed after migration:

$ php artisan migrate --seed

### Route model binding
Binding a route key to an underlying model when defining a route:

In [ ]:
// web.php
## Automatically finds a post with a given ID
Route::get('/posts/{post}', function (Post $post){
    return view('post', [
        'post'=> $post
    ])
})

Route model binding automatically defaults to finding a given post by ID, but can also specify which column to search by:

In [ ]:
## Finds by author
Route::get('/posts/{post:author}', function (Post $post){
    return view('post', [
        'post'=> $post
    ])
})

In a model, you can also manually define the default key to use in route model binding like so: 

In [ ]:
## app/Models/Post

class Post extends Model {
    //...
    public function getRouteKeyName()
    {
        return 'author'
    };
}

### Eloquent relationships
How to create relationships between different tables. For example, you want to add different categories blog posts can be in. You make a new table called Categories, and each blog post is assigned a category. 

Want to add a *foreign key* to your Post schema to relate it to the categories table: 

In [ ]:
#database/migrations/create_categories_table.php
//...
Schema::create('posts', function (Blueprint $table){
    /...
    $table->foreignId('category_id')
})

Then can configure it to show the relevant relationship based on the id. Laravel allows you to define the relationship between a given show and another table: 
- hasOne
- hasMany
- belongTo
- BelongToMany

In [ ]:
# App/Models/Post 
class Post extends Model
{
    ///...
    public function category()
    {
        return $this->belongsTo(Category::class);
    }
}

In [ ]:
$post = Post::first()
        
## Access as a category
$post->category;

Can specify the column name you want to use by passing a second argument to the belongsTo function:

In [ ]:
# App/Models/Post 
class Post extends Model
{
    ///...
    public function category()
    {
        return $this->belongsTo(Category::class, 'category_id');
    }
}

Then you can do the same for the Category model: a category hasMany blogs. 

### Eager loading
Contrast to lazy loading, where we make one query each time we need something from the DB. Eager loading makes multiple queries upfront to get all the data we need.

Example: Loading the latest blog post right away with its categories

In [ ]:
# routes/web.php
Route::get("/", function () {
    return view('posts', [
        'posts' => Post::latest()::with('category', 'user')->get()
    ])
})

With **Route model binding**

In [ ]:
# routes/web.php
Route::get("/posts/{id}", function (Post $post) {
    return view('posts', [
        'posts' => $post->load('category')
    ])
})

## Search
In Laravel, we can wrap a form in a get request and it will automatically send the request on submit

In [ ]:
<form  method="GET" action="#">
        <input 
                type="text"
                name="search" 
                value="{{ request('search') ?? ' ' }}"
            />
        <button type='submit'>search</button>
    </form>

The search contents are then sent via *request('search')*. In the backend, can use a SQL query with laravel's 'where'

In [ ]:
## Handle searching
Route::get('/', function () {
    $posts = Post::latest();
    if (request('search')) {
        $posts
            ->where('title', 'like', '%'.request('search').'%');
            ->orWhere('body', 'like', '%'.request('search').'%');
    }
    return view('posts', [
        'posts'=> $posts->get();
    ])
});

**Cleaner method:** Refactor all the logic out into its own controller, and then route through the controller:

In [ ]:
## app/http/controllers/BlogController
class BlogController extends Controller
{

    public function index()
    {
        $blogs = Blog::with('category', 'user');

        if(request('search') ?? false){
            $blogs->where('title', 'like', '%'.request('search').'%');
        }

        return view('main', [
        'blogs' => $blogs->get()
    ]);
    }
    
    public function show(Blog $blog)
    {
        return view('post', [
            'blog' => $blog->load('category', 'user')
        ]);
    }
}

Then simply route through this controller:

In [ ]:
#routes/web.php
Route::get('/', [PostController::class, 'index'])->name('home');
Route::get('/post/{blog}', [PostController::class, 'show'])

We can make this even cleaner by creating a scope directly on the eloquent models. To define a scope, prefix the function name with 'scope'.

You then can build your query and call the scope function afterwards: 
ex: *Blog::recent()->filter*

In [ ]:
class Post extends Model
{
    //...
    public function scopeFilter($query, $filters)
    {
        ## Call  Post::filter()
        # Takes query filter: Post::newQuery()->filter()
        if (filters('search') ?? false) {
        $query
            ->where('title', 'like', '%'.request('search').'%');
            ->orWhere('body', 'like', '%'.request('search').'%');
    }
    }
}

In [ ]:
# App\Http\Controllers\PostController
class PostController extends Controller
{
    public function idnex()
    {
        return view('posts', [
            'post' => Post::latest()->filter(request(['search', 'category']))->get()
        ]);
    }
}

**When** method allows you to call a method when a given value is true

In [ ]:
class Post extends Model
{
    //...
    public function scopeFilter($query, $filters)
    {
        ## Call  Post::filter()
        # Takes query filter: Post::newQuery()->filter()
        $query->when($filters['search'] ?? false, function ($query, $search){
            $query
            ->where('title', 'like', '%'.request('search').'%');
            ->orWhere('body', 'like', '%'.request('search').'%');
        })
        
        $query->when($filters['category'] ?? false, function ($query, $category){
            $query->whereHas('category', function($query) use ($category){
                $query->where('name', $category);
            });
            });
    }
}

## Controllers

In [ ]:
$ php artisan make:controller PostController

In [ ]:
## namespace App\Http\Controllers;

use App\Models\Blog;
use App\Models\Category;

use Illuminate\Http\Request;

class BlogController extends Controller
{
    public function index()
    {
        $blogs = Blog::with('category', 'user')->get();
        return view('main', [
        'blogs' => $blogs
    ]);

    }
}


In [ ]:
## Call a controller in the route
# Second argument is name of controller accent want to trigger
Route::get('/', [BlogController::class, 'index'])->name('home');

By Naming the route in this way, we can then just call the route like this: 

In [ ]:
route('home')

## Forms

In the html specify the method and the route. The name of the input is the name. 

In [ ]:
<form method="POST" action="/signup">
        @csrf
        <p><input name="name" id="name" type="text" placeholder="name"/></p>
        <p><input name="username" id="username" type="text" placeholder="username"/></p>
        <p><input name="email" id="email" type="email" placeholder="email"/></p>
        <p><input name="password" type="password" id="password" placeholder="password"/></p>
        <button type="submit">Signup</button>
</form>

**CSRF Attack** (cross site request forgery): A fake request from one site to another site. Stopping people make forms from another website that cause user changes on yours. Ex: Someone making a form on another website that submits a logout request to overton.io.

If you include @csrf in the controller function, laravel will automatically add a hidden input with a token that verifies the request is coming from the correct app. This validation helps prevent csrf attacks. 

In [ ]:
## Signup controller: App\Http\Controllers\RegistryController
class RegistryController extends Controller
{
    public function create()
    {
        return view('signup');
    }

    public function store()
    {
        # Validate
        $attributes = request()->validate([
            'name'=>array('required', 'max:255'),
            'username'=>array('required', 'max:255', Rule::unique('users', 'username')),
            'email'=>array('required', 'max:255', 'email'),
            'password'=>array('required', 'max:255', 'min:7')
        ]);
        
        $attributes['password'] = bcrypt($attributes['password'])
                
         # Create the user
        User::create($attributes);
        
        ## Log user in 
        auth()->login($user);
        
        return redirect('/')->with('success', 'Your account has been created');
    }
}

If the validation fails laravel will redirect back to the previous page. 

Lastly, when making the user, any attributes that are not fillable in the model will fail. To get around this, you can delete the fillable variable in the model and instead set $guarded to an empty array. This takes control away from laravel so it can be handled manually. 

In [ ]:
## app\models\user
class User extends Authenticatable
{
    //...
    ## protected $fillable=[name, email, password]
    protected $guarded = []
}

In [ ]:
## routes/web.php
Route::get('/signup', [RegistryController::class, 'create']);
Route::post('/signup',[RegistryController::class, 'store'] );

### Hashing passwords
Use the above function bcrypt to hash a password when passed in. We can then validate it using the following helper function.

In [ ]:
Illuminate\Support\Facades\Hash::check('password', $user->passwordb)

#### Eloquent mutator
Can define functions within an eloquent model (named set_nameofattribute_Attribute()), which will modify the attribute accordingly when a new attribute is passed in before saving to the db.

In [ ]:
## App\Models\User
public function setPasswordAttribute($password){
    $this->attributes['password'] = bcrypt($password)
}

#### Eloquent Accessor
Inverse of a mutator, this modifies an attribute upon getting it from the database. 

In [ ]:
public function getNameAttribute($name){
    # Set first letter to uppercase
    return ucfirst($name)
}

### Validation Errors
When validation fails laravel redirects back to the previous page and populates an errors variable.

We can read the variable directly in the HTML with the error directive. When inside an error directive, you have access to the message variable. 

In [ ]:
## views/signup.blade.php
/... 
@error('name')
    <p> {{ $message }} </p>
@enderror

## To show all errors at one: 
{{ $errors->all() }}

To avoid the user losing all their filled in values on redirect (annoying), can set the default value to any flashed session data from the previous request. On redirect, laravel will automatically flash this old session data. 

In [ ]:
<input type='text' name='name' value="{{ old('name') }}"

To handle errors being thrown by the backend, for example needing a unique name, we must ensure the validation matches the requirements of the backend. 

**Unique attributes**
The unique validation requirement takes the form: 'unique:tablename,columnname'- it checks with the backend that it is unique. Or in array form:

In [ ]:
## must be unique in table users, column username
$attributes = request() -> validate([
    'username' => array('required', Rule::unique('users', 'username'))
])

### Flash messages
Message to flash to user before redirect. Example, a success message. Shown in the example above, you include this in your controller.

In [ ]:
session()->flash('success', 'success!')

Then you can include the session message in your html:

In [ ]:
## On redirect page   
<?php if(session()->has('success')): ?>
    <div>
        <p><?= session('success') ?></p>
    </div>
<?php endif; ?>

### Middleware
App/http/kernel defines which middleware are global and which are route specific. Laravel has lots of inbuilt middleware. 

Let's say we want to restrict our signup page for only people who aren't signed in, we can restrict the link to only be shown to non-users. Use the middlewar 'guest' which restricts to people who aren't logged in. Inversely, 'auth' is for people who are logged in.

In [ ]:
## routes/web.php
Route::get('/signup', [RegistryController::class, 'create'])->middleware('guest');

Route::post('/signup',[RegistryController::class, 'store'])->middleware('guest');

Can then conditionally render the register button by using the @guest tag in our html:

In [ ]:
@guest
    <button href='/signup'>register</button>
@endguest

When a user is logged in, we can use the auth() helper to access their details:

In [ ]:
<div>Welcome, {{ auth()->user()->name }}</div>

### Login and out route

In [ ]:
## web.php
# Use generic sessions controller
Route::post('logout', [SessionsController::class, 'destroy'])->middleware('auth')

Route::post('logout', [SessionsController::class, 'create'])->middleware('guest')

Route::post('sessions', [SessionsController::class, 'store'])->middleware('guest')

In [ ]:
#App\http\controllers\SessionController

public function destroy()
{
    auth()->logout();
    return redirect('/')->with('success', 'Goodbye!')
}

public function create(){
    return view('login')
}

public function store()
{
    ## Check user exists
    request()->validate([
        'email'=>array('required', Rule::Exists('users', 'email')),
        'password'=>array('required')
    ])
    
    ## attempt to log in
    if(auth()->attempt($attributes)){
        return redirect('/')->with('success', 'welcome back!')
    }
    
    ## Otherwise redirect back with custom errors
    return back()
        ->withInput() # Save input details
        ->withErrors(['email'=>'Incorrect email']);
    
    
    ## ALTERNATIVELY can throw a validation error:
    # does the same thing
    throw ValidationException::withMessages([
        'email'=>'Your email was incorrect'
    ])
}

In [ ]:
## html form post to sessions
<form method="POST" action="/sessions">
//... login
</form>

## Auth errors: 
Note if it is throwing issues when you try to use auth, you will need to go into config/auth.php and update the pathway to user (you moved it into models folder)

## CRUD api set up
Below command makes a controller with the five key methods: 
- index
- store
- update
- create
- destroy
- (show)

In [ ]:
$ php artisan make:controller Api/UserController --api --model=User

Use of apiResource() when defining your route allows you to define all your key routes in one single line. apiResource() maps to all the API methods in the controller. 

In [ ]:
## routes/api.php
Route::apiResource('users', UserController::class);

Can use a resource (a separate class) to ensure returns are formatted correctly. 

In [ ]:
$ php artisan make:resource UserGroupResource

In [ ]:
## App\http\resources\usergroupresources.php
# What to return from request
class UserGroupResource extends JsonResource
{
    /**
     * Transform the resource into an array.
     *
     * @return array<string, mixed>
     */
    public function toArray(Request $request): array
    {
        return [
            "group_id" => $this->group_id,
            "invite_code" => $this->invite_code,
            "title" => $this->title,
            "owner" => $this->owner,
            "default_flags" => $this->default_flags,
            "created_by" => $this->created_by,
            "added_on" => $this->added_on,
            "hibernated" => $this->hibernated,
            "users_count" => $this->whenCounted("users")
        ];
    }
}

Can then use in the controller. Use **collection** function to call all user groups with users in them from the db. 

Below returns all user groups as data object

#### Index

In [ ]:
## app\http\controllers\usergroupcontroller.php
public function index()
    {
        Gate::authorize("has-flag", ["admin"]);

        return UserGroupResource::collection(UserGroup::all())
    }

Can also call the **paginate** method to return an input number of items with links to the next page's api:

In [ ]:
UserGroupResouce::collection(UserGroup::paginate(5))  # 5 per page

#### Show

In [ ]:
public function show(UserGroup $usergroup)
{
    ## Returns new resource with object passed in
    return new UserGroupResouce($usergroup);
}

#### Store

In [ ]:
public function store(Request $request)
{
    $user = $request->validate([
        'name'=> array('required', 'string', 'max:255'),
        'email'=> array('required', 'email', 'unique:users'),
        'password'=> array('required', 'min:7', 'confirmed')
    ]);
    
    User::create($user)
    
    ## Hashed in eloquent model
    
    return new UserResource($user)
}

##### Store user requests
Can make another request (app/http/requests) which is already populated with authorize() and rules() method. Requests offer an object-oriented way of interacting with current HTTP requests being handled by your app, as well as retrieve input, cookies and files that are submitted with the request. 

Automatically will validate the request agaisnt the rules function

In [ ]:
$ php artisan make:request StoreUserRequest

In [ ]:
## App\http\requests\storeusergrouprequest

class StoreUserGroupRequest extends FormRequest
{
    /**
     * Determine if the user is authorized to make this request.
     */
    public function authorize(): bool
    {
        return Gate::allows("has-flag", ["admin"]);
    }

    /**
     * Get the validation rules that apply to the request.
     *
     * @return array<string, \Illuminate\Contracts\Validation\Rule|array|string>
     */
    public function rules(): array
    {
        return [
            'group_id' => array('required', 'string', 'max:255', 'unique:user_groups'),
            'title' => array('required', 'string', 'max:255'),
            'owner' => array('required', 'string', 'max:255'),
            'default_flags' => array('required', 'array', 'min:1'),
            'hibernated' => array('bool')
        ];
    }
}

This will automatically validate the request agaisnt the given rules. We just need to import it in the resource class's argument: 

In [ ]:
public function store(StoreUserRequest $request)
{
    $user = $request->validate([
        'name'=> array('required', 'string', 'max:255'),
        'email'=> array('required', 'email', 'unique:users'),
        'password'=> array('required', 'min:7', 'confirmed')
    ]);
    
    User::create($user)
    
    ## Hashed in eloquent model
    
    return new UserResource($user)
}

#### Update

In [ ]:
## make another request object
## app\http\request\UpdateUserRequest.php
## Can copy-paste validation from create user group

class StoreUserGroupRequest extends FormRequest
{
    /**
     * Determine if the user is authorized to make this request.
     */
    public function authorize(): bool
    {
        return Gate::allows("has-flag", ["admin"]);
    }

    /**
     * Get the validation rules that apply to the request.
     *
     * @return array<string, \Illuminate\Contracts\Validation\Rule|array|string>
     */
    public function rules(): array
    {
        return [
            'group_id' => array('required', 'string', 'max:255', 'unique:user_groups'),
            'title' => array('required', 'string', 'max:255'),
            'owner' => array('required', 'string', 'max:255'),
            'default_flags' => array('required', 'array', 'min:1'),
            'hibernated' => array('bool')
        ];
    }
}

In [ ]:
## Resource class
public function update(UpdateUserRequest, $request, User $user)
{
    $user->update([
        'name' => $request->name,
        'email'=> $request->email,
        'password'=> Hash::make($request->password)
    ])
    
    return new UserResource($user)
}

#### Destroy

In [ ]:
public function destroy(User $user)
{
    return $user->delete()
}